# Dataframe - Higher level API auf RDDs

Wir haben uns vorhin RDDs angeschaut. Diese bieten uns schon eine Reihe nützlicher Feature an. Viele Optimierungen müssen wir aber von Hand vornehmen.

Wir werden sehen, was dass wir mit Dataframe einfach nur einfacher und eleganter entwickeln können, sondern das der Code viel schneller ist, als nicht optimierter RDD Code.

Wir werden ganz praktisch:

* Sehen wie wir Dataframes erzeugen können
* Schemata auf Daten anwenden
* Daten analysieren und bereinigen
* Transformationen anwenden

[API-Doc](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html)

## Spark Optimierungen auf Dataframes

Es gibt zwei wesentliche Ansätze an denen Spark Optimierungen vornimmt

![](spark-optimierungen.dio.svg)


## [Catalyst Optimierer](https://www.databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)

Der Catalyst optimiert den Ausführungsplan. Wer sich mal bei PostgreSQL mit [_explain_](https://www.postgresql.org/docs/current/sql-explain.html) den Plan für sein Statement angeschaut hat, der hat schon eine Idee was der Catalyst macht.

![](optimisation-phases.svg)
### Analyse-Phase

1. Syntax Check
1. Erzeuge: Unresolved logical Plan (unresolved, weil weder geprüft wird ob referenziert Spalten existieren, noch ob sie den richtigen Typ haben...)
1. Nun wird mit Hilfe des Cataloges (Verzeichniss aller Spalten/Tabellen/Datentypen) der logische Plan erzeugt. Jetzt ist geklärt ob die Spalten existieren und passen...

### Logische Optimierung

  * [Predicate Pushdown](https://medium.com/microsoftazure/data-at-scale-learn-how-predicate-pushdown-will-save-you-money-7063b80878d7)
  * [Constant folding](https://en.wikipedia.org/wiki/Constant_folding)
  * ...

Dies führt zu optimierten logischen Plan.

### Physical Planning

Für den optimierten logsichen Plan werden ein Reihe von physischen Plänen berechnet und mit einem Kostenmodell bewertet. Der Günstige wird dann genommen.

### Code Generation

Am Ende sind es wieder RDDs die in Java Bytecode überführt werden.

### Beachte

* All diese Optimierungen bekommst Du nicht, wenn Du direkt RDD Code schreibst. Diese müsstest Du selbst finden und ausprogrammieren.
* Dies führt auch dazu, dass Dataframe-Code in der Regel nahezu so schnell ist wie handoptimierter RDD-Code. (überlege selber ob Du diese Optimierungen überhaupt selber schreiben kannst oder willst)
* Das führt ebenfalls dazu, dass Python/R/SQL/Scala-Dataframe-Code gleichschnell sind, weil sie alle in RDD Code überführt werden.



In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("catalyst")
        .master("local[4]")  
        .config("spark.dynamicAllocation.enabled",False)
        .config("spark.sql.adaptive.enabled",False)
        .getOrCreate()
)
spark

23/09/10 08:53:07 WARN Utils: Your hostname, keen-northcutt resolves to a loopback address: 127.0.1.1; using 116.203.107.225 instead (on interface eth0)
23/09/10 08:53:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/10 08:53:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import schemata
taxi_rides_df = spark.read.csv("YellowTaxis_202210.csv.gz", header = True, schema=schemata.yellow_taxi_schema)

taxi_rides_by_location = taxi_rides_df.groupby("PULocationID").count()

filtered_taxi_rides_by_location = taxi_rides_by_location.where("PULocationID == 81")

taxi_rides_by_location.show()

+------------+-----+
|PULocationID|count|
+------------+-----+
|         148|38049|
|         243|  514|
|          31|   22|
|         137|43171|
|         251|    6|
|          85|   95|
|          65| 3042|
|         255| 1969|
|          53|   41|
|         133|   92|
|          78|  113|
|         155|  127|
|         108|   78|
|         211|26838|
|         193| 3383|
|          34|  113|
|         126|   67|
|         101|   50|
|         115|    5|
|          81|   73|
+------------+-----+
only showing top 20 rows



![](different-concrete-plans.dio.svg)